In [13]:
!pip install -r requirements.txt

In [11]:
%cd serve_bert/example

[WinError 3] Системе не удается найти указанный путь: 'serve_bert/example'
C:\Users\Artyom\serve_bert\example


In [5]:
%pwd

'C:\\Users\\Artyom\\serve_bert\\example'

In [4]:
from transformers import AutoModel

model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

# Push the model to your namespace with the name "my-finetuned-bert".
model.save_pretrained("my-finetuned-bert")

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

C:\Users\Artyom\anaconda3\lib\site-packages\huggingface_hub\file_download.py:125: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Artyom\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
model_store_dir = "./model_store"
serialized = "./my-finetuned-bert/pytorch_model.bin"
model_config = "./my-finetuned-bert/config.json"

In [9]:
!torch-model-archiver --model-name lit_sentiment_model \
--version 0.1 \
--serialized-file "$serialized" \
--handler ../serving/serve_pretrained.py \
--runtime python3 \
--extra-files "$model_config" \
--export-path "$model_store_dir"

In [10]:
!torchserve --start --ncs --model-store "$model_store_dir" --models sentiment_model=lit_sentiment_model.mar --ts-config ./torchserve.config --foreground

2022-11-22T22:36:55,430 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2022-11-22T22:36:55,696 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.6.1
TS Home: C:\Users\Artyom\Anaconda3\Lib\site-packages
Current directory: C:\Users\Artyom\serve_bert\example
Temp directory: C:\Users\Artyom\AppData\Local\Temp
Metrics config path: C:\Users\Artyom\Anaconda3\Lib\site-packages/ts/configs/metrics.yaml
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 1522 M
Python executable: C:\Users\Artyom\anaconda3\python.exe
Config file: ./torchserve.config
Inference address: http://127.0.0.1:7863
Management address: http://127.0.0.1:8863
Metrics address: http://127.0.0.1:8864
Model Store: C:\Users\Artyom\serve_bert\example\model_store
Initial Models: sentiment_model=lit_sentiment_model.mar
Log dir: C:\Users\Artyom\serve_bert\example\logs
Metrics dir: C:\Users\Artyom\serve_bert\example\logs
Netty threads: 0
Netty client threads: 0
Default w

java.io.IOException: Failed to bind to address: http://127.0.0.1:7863
	at org.pytorch.serve.ModelServer.initializeServer(ModelServer.java:311)
	at org.pytorch.serve.ModelServer.startRESTserver(ModelServer.java:372)
	at org.pytorch.serve.ModelServer.startAndWait(ModelServer.java:117)
	at org.pytorch.serve.ModelServer.main(ModelServer.java:98)
Caused by: java.net.BindException: Address already in use: bind
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:555)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.netBind(ServerSocketChannelImpl.java:344)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:301)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:134)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:550)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandle

In [ ]:
!streamlit run serving/lit_sent_bert.py --server.port 7864